In [4]:
import pandas as pd
import requests
import json
import time
import gender_guesser.detector as gender
from babel import Locale
import pycountry_convert as pycc
from babel import Locale
sleep=0.4

In [8]:
LOCAL=False
if LOCAL:
    f=open('data/inspire_LA.json','r')
    df=json.load(f)
    f.close()
else:
    r=requests.get('https://github.com/restrepo/inspireauth/raw/main/data/inspire_LA.json')
    df=r.json()

In [9]:
print(len(df))

57588


## Extended papers data structure including `rank` and related information
1) Get ranks for each institution for authors with declared profile
2) Get ranks for each paper

### Extend data-scheme

In [10]:
from datetime import datetime
import copy
def get_country(institution_id,df):
    institution_id = str(institution_id)
    for d in df:
        country = None
        if d.get('institution_id') == institution_id:
            country = d.get('country')
            break
    return country
def flatten_list(L):
    return [item for sublist in L for item in sublist]

In [11]:
UPDATE = False
if UPDATE:
    #Extend 'positions' data-scheme
    i=0
    for d in df:
        if d.get('positions'):
            for p in d.get('positions'):
                try:
                    p['institution_id']=p.get('record').get('$ref').split('/')[-1]
                except:
                    p['institution_id']=None
                if p.get('institution_id'):
                    p['country']=get_country(p.get('institution_id'),df)
                else:
                    p['country']=None
                p['author_id']=d.get('author_id')
        print(str(i).zfill(5),end='\r')
        i += 1                
    
    #Extend 'papers' data-scheme
    #d=df[4722]
    i=0
    for d in df:
        if d.get('positions'):
            dp=copy.deepcopy(d.get('positions'))
            for p in dp:
                try:
                    p['institution_id'] = p.get('record').get('$ref').split('/')[-1]
                except:
                    p['institution_id'] = None
    
        il = [p.get('institution_id') for p in dp]
        #papers = copy.deepcopy(d.get('papers'))
        papers = d.get('papers')
        for pr in papers:
            # Fix institution
            pr['institution_id'] = pr.pop('instituion_id')
            if pr.get('institution_id') in il:
                end_date=None
                ps=[p for p in dp if p.get('institution_id') == pr.get('institution_id')]
                for p in ps:
                    if p.get('end_date'):
                        end_date=int(p.get('end_date').split('-')[0])
                    else:
                        if p.get('current'):
                            end_date=datetime.now().year+1
                    start_date=p.get('start_date')
                    if start_date:
                        start_date=int(start_date.split('-')[0])
                    try:
                        year=eval(pr.get('year'))
                    except:
                        year=pr.get('year')
                    if year:
                        year=int(year)
                    if year and start_date and end_date and year in range(start_date,end_date+2): #+1 of tolerance
                        pr['rank'] = p.get('rank')
                        pr['start_date'] = p.get('start_date')
                        pr['end_date'] = p.get('end_date')
                        if p.get('current'):
                            pr['current'] = p.get('current')
                        pr['institution'] =p.get('institution')
                        pr['country'] = get_country(p.get('institution_id'),df)
            #if i==3:
            #    raise Exception(f'break -> {i}')
        print(str(i).zfill(5),end='\r')
        i += 1

    f=open('inspire_LA.json','w')
    json.dump(df,f)
    f.close()

### Find gender
See [gender.ipynb](/.gender.ipynb)

### Rank of the author analysis
Rank(s) obtained by the author in the country

In [12]:
profile_ranks = [d.get('positions') for d in df if d.get('positions') and 
                        any([True for dd in d.get('positions') if dd.get('rank')])]
profile_ranks = flatten_list(profile_ranks)
profile_ranks = [p for p in profile_ranks if p.get('rank')]

In [13]:
dfp=pd.DataFrame(profile_ranks).drop_duplicates(subset=[
     'rank','end_date','start_date','institution','curated_relation','institution_id','country','author_id','current','hidden'
    ]).reset_index(drop=True)
dfp['rank'].value_counts()

rank
POSTDOC          6455
PHD              6241
SENIOR           3490
UNDERGRADUATE    3128
JUNIOR           1491
MASTER           1351
VISITOR           708
STAFF             548
OTHER              20
Name: count, dtype: int64

In [14]:
len(df)

57588

In [15]:
dfp.shape,dfp['rank'].value_counts().sum()

((23432, 11), np.int64(23432))

In [16]:
dfp['rank'].unique()

array(['PHD', 'SENIOR', 'POSTDOC', 'MASTER', 'UNDERGRADUATE', 'JUNIOR',
       'VISITOR', 'STAFF', 'OTHER'], dtype=object)

In [17]:
dfp[dfp['country']=='Chile']['rank'].value_counts()

rank
POSTDOC          110
PHD              108
UNDERGRADUATE     77
SENIOR            65
MASTER            46
JUNIOR            44
VISITOR           11
STAFF              4
OTHER              3
Name: count, dtype: int64

Explanation: 
* 110 authors work as POSTDOC in Chile, etc.
* 108 authors did their PHD in Chile, etc.

In [18]:
tmp=dfp[dfp['country']=='Chile']
tmp[ (tmp['rank']=='STAFF') ]

,rank,record,end_date,start_date,institution,curated_relation,institution_id,country,author_id,current,hidden
2973,STAFF,{'$ref': 'https://inspirehep.net/api/instituti...,NaN,2016,"Chile Austral U., Valdivia",True,903797,Chile,P.Mella.2,True,NaN
7681,STAFF,{'$ref': 'https://inspirehep.net/api/instituti...,NaN,2019,"San Sebastian U., Chile",True,1459222,Chile,M.Riquelme.1,True,NaN
23270,STAFF,{'$ref': 'https://inspirehep.net/api/instituti...,2019,2016,Concepcion U.,True,904435,Chile,F.I.Portales.Oliva.1,False,False
23271,STAFF,{'$ref': 'https://inspirehep.net/api/instituti...,2019,2016,"San Sebastian U., Chile",True,1459222,Chile,F.I.Portales.Oliva.1,False,False


In [19]:
dfp[dfp['country']=='Brazil']['rank'].value_counts()

rank
PHD              445
POSTDOC          345
UNDERGRADUATE    254
MASTER           239
SENIOR           232
JUNIOR            65
VISITOR           51
STAFF             46
Name: count, dtype: int64

In [20]:
dfp[dfp['country']=='Argentina']['rank'].value_counts()

rank
PHD              136
UNDERGRADUATE     88
SENIOR            42
POSTDOC           41
MASTER            27
JUNIOR            16
STAFF             13
VISITOR            3
Name: count, dtype: int64

In [21]:
dfp[dfp['country']=='Mexico']['rank'].value_counts()

rank
PHD              185
POSTDOC          124
SENIOR            87
UNDERGRADUATE     85
MASTER            71
JUNIOR            59
STAFF             26
VISITOR           10
OTHER              1
Name: count, dtype: int64

In [22]:
dfp[dfp['country']=='Colombia']['rank'].value_counts()

rank
UNDERGRADUATE    48
SENIOR           32
MASTER           21
JUNIOR           19
POSTDOC          15
PHD              15
STAFF             5
VISITOR           1
Name: count, dtype: int64

In [23]:
dfp[dfp['country']=='Uruguay']['rank'].value_counts()

rank
UNDERGRADUATE    5
MASTER           3
PHD              2
STAFF            2
POSTDOC          1
SENIOR           1
JUNIOR           1
Name: count, dtype: int64

In [24]:
dfp[dfp['country']=='Uruguay']

,rank,record,end_date,start_date,institution,curated_relation,institution_id,country,author_id,current,hidden
543,MASTER,{'$ref': 'https://inspirehep.net/api/instituti...,2003,NaN,"Republica U., Montevideo",True,903022,Uruguay,R.A.Porto.1,NaN,NaN
845,POSTDOC,{'$ref': 'https://inspirehep.net/api/instituti...,2014,2012,"Republica U., Montevideo",True,903022,Uruguay,J.Olmedo.1,NaN,NaN
857,PHD,{'$ref': 'https://inspirehep.net/api/instituti...,2012,2007,"Republica U., Montevideo",True,903022,Uruguay,S.Rastgoo.1,False,False
896,STAFF,{'$ref': 'https://inspirehep.net/api/instituti...,NaN,NaN,"Republica U., Montevideo",True,903022,Uruguay,G.A.Gonzalez.Sprinberg.1,True,NaN
1347,MASTER,{'$ref': 'https://inspirehep.net/api/instituti...,2012,2009,"Republica U., Montevideo",True,903022,Uruguay,L.Duarte.1,NaN,NaN
1348,UNDERGRADUATE,{'$ref': 'https://inspirehep.net/api/instituti...,2008,2002,"Republica U., Montevideo",True,903022,Uruguay,L.Duarte.1,NaN,NaN
1415,SENIOR,{'$ref': 'https://inspirehep.net/api/instituti...,NaN,2008,"Republica U., Montevideo",True,903022,Uruguay,R.Donangelo.1,NaN,NaN
1417,PHD,{'$ref': 'https://inspirehep.net/api/instituti...,2003,NaN,"Republica U., Montevideo",True,903022,Uruguay,Pablo.Mora.1,NaN,NaN
1433,MASTER,{'$ref': 'https://inspirehep.net/api/instituti...,NaN,NaN,"Republica U., Montevideo",True,903022,Uruguay,M.Ponce.1,NaN,NaN
1434,UNDERGRADUATE,{'$ref': 'https://inspirehep.net/api/instituti...,NaN,NaN,"Republica U., Montevideo",True,903022,Uruguay,M.Ponce.1,NaN,NaN


In [25]:
pd.DataFrame(df)[50:60][['profile_id','author_id','full_name','name','country','gender']]

,profile_id,author_id,full_name,name,country,gender
50,None,J.Chela.Flores.1,"Chela-Flores, J.",None,Italy,unknown
51,2074632,R.Paredes.1,"Paredes, R.",None,Venezuela,unknown
52,1019827,C.Aragone.1,"Aragone, C.","{'value': 'Aragone, Carlos', 'preferred_name':...",Venezuela,male
53,2455877,G.Colagiacomo.1,"Colagiacomo, G.",None,Venezuela,unknown
54,1008438,R.Gianvittorio.1,"Gianvittorio, R.","{'value': 'Gianvittorio, Rita', 'preferred_nam...",Venezuela,female
55,1000877,L.Leal.1,"Leal, L.","{'value': 'Leal, Lorenzo', 'preferred_name': '...",Venezuela,male
56,1008978,R.Gambini.1,"Gambini, Rodolfo","{'value': 'Gambini, Rodolfo', 'preferred_name'...",Uruguay,male
57,1050545,A.Trias.1,"Trias, Antoni","{'value': 'Trias, Antoni', 'preferred_name': '...",Spain,male
58,2222068,E.Mac.2,"Mac, E.",None,Venezuela,unknown
59,985291,E.Ugaz.1,"Ugaz, E.","{'value': 'Ugaz, Eduardo', 'preferred_name': '...",Peru,male


### Gender analysis

In [26]:
dfm = [d for d in df if d.get('gender')=='male' ]
dff = [d for d in df if d.get('gender')=='female' ]

In [27]:
profile_ranks = [d.get('positions') for d in dfm if d.get('positions') and 
                        any([True for dd in d.get('positions') if dd.get('rank')])]
profile_ranks = flatten_list(profile_ranks)
profile_ranks = [p for p in profile_ranks if p.get('rank')]

dfp=pd.DataFrame(profile_ranks).drop_duplicates(subset=[
     'rank','end_date','start_date','institution','curated_relation','institution_id','country','author_id','current','hidden'
    ]).reset_index(drop=True)
dfp['rank'].value_counts()

rank
POSTDOC          4768
PHD              4661
SENIOR           2755
UNDERGRADUATE    2404
JUNIOR           1081
MASTER            978
VISITOR           523
STAFF             443
OTHER              14
Name: count, dtype: int64

In [28]:
dfp[dfp['country']=='Brazil']['rank'].value_counts()

rank
PHD              317
POSTDOC          245
UNDERGRADUATE    184
SENIOR           182
MASTER           172
JUNIOR            48
STAFF             35
VISITOR           35
Name: count, dtype: int64

In [29]:
dfp[dfp['country']=='Colombia']['rank'].value_counts()

rank
UNDERGRADUATE    45
SENIOR           27
MASTER           19
JUNIOR           17
POSTDOC          14
PHD              14
STAFF             4
VISITOR           1
Name: count, dtype: int64

In [30]:
profile_ranks = [d.get('positions') for d in dff if d.get('positions') and 
                        any([True for dd in d.get('positions') if dd.get('rank')])]
profile_ranks = flatten_list(profile_ranks)
profile_ranks = [p for p in profile_ranks if p.get('rank')]

dfp=pd.DataFrame(profile_ranks).drop_duplicates(subset=[
     'rank','end_date','start_date','institution','curated_relation','institution_id','country','author_id','current','hidden'
    ]).reset_index(drop=True)
dfp['rank'].value_counts()

rank
POSTDOC          749
PHD              717
SENIOR           316
UNDERGRADUATE    305
JUNIOR           166
MASTER           132
STAFF             62
VISITOR           58
OTHER              3
Name: count, dtype: int64

In [31]:
dfp[dfp['country']=='Brazil']['rank'].value_counts()

rank
PHD              56
POSTDOC          45
UNDERGRADUATE    31
MASTER           24
SENIOR           17
VISITOR           6
STAFF             5
JUNIOR            4
Name: count, dtype: int64

In [32]:
dfp[dfp['country']=='Colombia']['rank'].value_counts()

rank
SENIOR    2
MASTER    1
Name: count, dtype: int64

In [33]:
dfp[dfp['country']=='Colombia']

,rank,record,current,start_date,institution,curated_relation,institution_id,country,author_id,end_date,hidden
160,SENIOR,{'$ref': 'https://inspirehep.net/api/instituti...,NaN,1999,Antonio Narino U.,True,908168,Colombia,M.Losada.1,2019,NaN
269,MASTER,{'$ref': 'https://inspirehep.net/api/instituti...,NaN,NaN,"Colombia, U. Natl.",True,906485,Colombia,Erika.Catano.Mur.1,2012,NaN
1195,SENIOR,{'$ref': 'https://inspirehep.net/api/instituti...,True,2007,Antonio Narino U.,True,908168,Colombia,G.A.Navarro.1,NaN,NaN


### Gender evolution with ranks

In [34]:
profile_ranks = [d.get('positions') for d in df if d.get('positions') and 
                        any([True for dd in d.get('positions') if dd.get('rank')])]
profile_ranks = flatten_list(profile_ranks)
profile_ranks = [p for p in profile_ranks if p.get('rank')]

In [35]:
DF=pd.DataFrame(df)

In [36]:
DF = DF.drop_duplicates('author_id').reset_index(drop=True)
DF.shape

(29969, 13)

In [37]:
ag = dict(zip(DF['author_id'].to_list(),DF['gender'].to_list()))

In [38]:
dfp=pd.DataFrame(profile_ranks).drop_duplicates(subset=[
     'rank','end_date','start_date','institution','curated_relation','institution_id','country','author_id','current','hidden'
    ]).reset_index(drop=True)
dfp['rank'].value_counts()

rank
POSTDOC          6455
PHD              6241
SENIOR           3490
UNDERGRADUATE    3128
JUNIOR           1491
MASTER           1351
VISITOR           708
STAFF             548
OTHER              20
Name: count, dtype: int64

In [39]:
len(dfp)

23432

In [40]:
dfp['gender'] = dfp['author_id'].apply(lambda a: ag[a] )

In [41]:
dfp['ranks'] = dfp.apply(lambda r: dfp[dfp['author_id'] == r['author_id'] ]['rank'].unique(),axis='columns')

In [42]:
dfp['ranks'] = dfp['ranks'].apply(lambda L: sorted(list(L)))

In [43]:
dfp['ranks'].iloc[0]

['PHD']

In [44]:
dfpu = dfp.drop_duplicates('author_id').reset_index(drop=True)

In [45]:
dfpu.shape

(7023, 13)

In [46]:
class get_gender:
    def __init__(self,df):
        self.df = df
        self.all = df.shape[0]
        self.male = df[df['gender'] == 'male'].shape[0]
        self.female = df[df['gender'] == 'female'].shape[0]
        self.unknown = df[ (df['gender'] != 'male') & (df['gender'] != 'female') ].shape[0]
    def __call__(self):
        return self.df

In [47]:
student = get_gender(dfpu)

In [48]:
student.all,student.male+student.female+student.unknown

(7023, 7023)

In [49]:
def gender_stats(n=0):
    All = dfpu[dfpu['ranks'].apply(lambda L: len(L) > n)]
    not_permanent = dfpu[dfpu['ranks'].apply(lambda L: 
           len(set(['SENIOR','JUNIOR','VISITOR', 'STAFF']
                  ).intersection(L)) == 0 and len(L) > n )]

    only_permanent = dfpu[dfpu['ranks'].apply(lambda L: 
          len( set(['PHD','POSTDOC','MASTER','UNDERGRADUATE','OTHER']
                  ).intersection(L))==0 and len(L) > n ) ]

    student = not_permanent[ not_permanent['ranks'].apply(lambda L: 'POSTDOC' not in L and len(L) > n) ]

    postdoc = not_permanent[ not_permanent['ranks'].apply(lambda L: 'POSTDOC' in L and len(L) > n )  ]

    full_career = dfpu[dfpu['ranks'].apply(lambda L: 
           len(set(['SENIOR','JUNIOR','VISITOR', 'STAFF']).intersection(L))>0 
      and len(set(['PHD','POSTDOC','MASTER','UNDERGRADUATE','OTHER']).intersection(L))>0
            and len(L) > n+1)] # have declared positions both as not_permanent and permanent

    permanent = pd.concat((only_permanent,full_career)).reset_index(drop=True) 
    prmnt = permanent # full_career #
    senior = prmnt[prmnt['ranks'].apply(lambda L: 
           len(set(['SENIOR']).intersection(L))>0
                                      )]
    junior = prmnt[prmnt['ranks'].apply(lambda L: 
           len(set(['SENIOR']).intersection(L))==0
                                      )]
    # CHECKED WITH:
    #permanent = dfpu[dfpu['ranks'].apply(lambda L: 
    #       len(set(['SENIOR','JUNIOR','VISITOR', 'STAFF']
    #              ).intersection(L)) != 0 and len(L) > n )]

    return { 'All':get_gender(All),
            'student':get_gender(student), 'postdoc':get_gender(postdoc), 
            'not_permanent':get_gender(not_permanent), 'junior':get_gender(junior),
            'senior':get_gender(senior),'only_permanent':get_gender(only_permanent),
            'full_career':get_gender(full_career), 'permanent':get_gender(permanent)
            }

In [50]:
tps = []
for n in range(2):
    gs = gender_stats(n)
    for tp in gs.keys():
        try:
            f_m = round(gs[tp].female/gs[tp].male*100,1)
        except:
            f_m = np.nan
        tps.append({'n': n+1,'type': tp ,'all': gs[tp].all,'gender':gs[tp].all-gs[tp].unknown,
                'male':gs[tp].male, 'female':gs[tp].female,
                '% f/m': f_m ,'df':gs[tp]()})

In [51]:
2238+3020,1440+2206

(5258, 3646)

In [52]:
fm = pd.DataFrame(tps).sort_values(['type','n']).reset_index(drop=True)
fm

,n,type,all,gender,male,female,% f/m,df
0,1,All,7023,6038,5258,780,14.8,rank ...
1,2,All,5354,4601,4013,588,14.7,rank ...
2,1,full_career,3336,2866,2528,338,13.4,rank ...
3,2,full_career,2852,2439,2161,278,12.9,rank ...
4,1,junior,1206,1005,859,146,17.0,rank ...
5,2,junior,887,730,631,99,15.7,rank ...
6,1,not_permanent,3054,2620,2238,382,17.1,rank ...
7,2,not_permanent,1962,1684,1440,244,16.9,rank ...
8,1,only_permanent,633,552,492,60,12.2,rank ...
9,2,only_permanent,56,51,45,6,13.3,rank ...


### Checks with n=1

In [ ]:
assert fm[fm['type']=='All'].iloc[0].gender == fm[fm['type']=='permanent'].iloc[0].gender + fm[fm['type']=='not_permanent'].iloc[0].gender

In [ ]:
assert fm[fm['type']=='not_permanent'].iloc[0].gender == fm[fm['type']=='student'].iloc[0].gender + fm[fm['type']=='postdoc'].iloc[0].gender

In [ ]:
assert fm[fm['type']=='permanent'].iloc[0].gender == fm[fm['type']=='senior'].iloc[0].gender + fm[fm['type']=='junior'].iloc[0].gender

In [ ]:
tmp=fm['df'].loc[2]
tmp[tmp['ranks'].apply(len)==1].shape

### Rank paper analysis
Rank of the authors of each paper. A paper can have a rank for each author in the best case scenario

In [ ]:
papers_ranks = [d.get('papers') for d in df if d.get('papers') and 
                        any([True for dd in d.get('papers') if dd.get('rank')])]
papers_ranks = flatten_list(papers_ranks)
papers_ranks = [p for p in papers_ranks if p.get('rank')]

In [ ]:
dfp=pd.DataFrame(papers_ranks)
dfp=dfp.drop_duplicates(subset=[
     'recid','year','citation_count','journal_id','author_id','institution_id','rank','start_date','end_date','institution','country','current'
    ]).reset_index(drop=True)

In [ ]:
dfp['rank'].value_counts()

In [ ]:
isinstance('1',str)

In [ ]:
dfpu['start_date'] = dfpu.start_date.str.split('-').str[0].apply(lambda d: eval(d) if isinstance(d,str) else 0)

In [ ]:
def count_f_m_year(d,df,year = 'start_date'):
    m=df[(df[year]==d) & (df['gender'] == 'male') ].shape[0]
    f=df[(df[year]==d) & (df['gender'] == 'female') ].shape[0]
    if m > 0:
        f_m = f/m 
    else:
        f_m = float('nan')
    return f_m

In [ ]:
dfpu['f/m_year'] = dfpu['start_date'].apply(lambda d: count_f_m_year(d,dfpu))

In [ ]:
kk = dfpu.drop_duplicates('start_date')
kk = kk.sort_values('start_date').reset_index(drop=True)
kk[kk['start_date'] == 1990]

In [ ]:
kk.loc[39,'f/m_year'] = 0.17

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(kk.start_date,kk['f/m_year']*100,'ko')
plt.plot(kk.start_date,kk['f/m_year']*100)
plt.xlim(1970,2024)
plt.ylim(-1,25)
plt.grid()
plt.xlabel('Start year in profile',size =15)
plt.ylabel('female/male [%]',size=15)
plt.savefig('f_m_evolution.pdf')

In [ ]:
dfp[dfp['country']=='Brazil']['rank'].value_counts()

Explanation: 5339 papers have a SENIOR author from Brazil at the moment of the publication

In [ ]:
dfp[dfp['country']=='Chile']['rank'].value_counts()

In [ ]:
dfp[dfp['country']=='Argentina']['rank'].value_counts()

In [ ]:
dfp[dfp['country']=='Colombia']['rank'].value_counts()

Explanation: 38 papers have a PHD author from Colombia at the moment of the publication

In [ ]:
tmp=dfp[dfp['country']=='Colombia']
tmp[tmp['rank']=='PHD']

### Rank author-papers analysis

In [ ]:
profile_ranks = [d.get('positions') for d in df if d.get('positions') and 
                        any([True for dd in d.get('positions') if dd.get('rank')])]
profile_ranks = flatten_list(profile_ranks)
profile_ranks = [p for p in profile_ranks if p.get('rank')]
dfp=pd.DataFrame(profile_ranks)

In [ ]:
papers_ranks = [d.get('papers') for d in df if d.get('papers') and 
                        any([True for dd in d.get('papers') if isinstance(dd.get('rank'),str)])]
papers_ranks = flatten_list(papers_ranks)
papers_ranks = [p for p in papers_ranks if p.get('rank')]

#### Author gender evolution

In [ ]:
dfpp=pd.DataFrame(papers_ranks)

In [ ]:
dfpp['gender'] = dfp['author_id'].apply(lambda a: ag[a] )

In [ ]:
dfpp['year'] = dfpp['year'].apply(int)

In [ ]:
dfpp['f/m_year'] = dfpp['year'].apply(lambda d: count_f_m_year(d,dfpp,year='year'))

In [ ]:
dfpp['f/m_year']

In [ ]:
#kk = dfpp.drop_duplicates('year')
kk = dfpp.sort_values('year').reset_index(drop=True)
#kk[kk['year'] == 1990]

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(kk.year,kk['f/m_year']*100,'ko')
plt.plot(kk.year,kk['f/m_year']*100)
plt.ylim(10,16)
plt.xlim(1999,2023)
#plt.title('paper counts > 600')
plt.grid()
plt.xlabel('Publication year',size =15)
plt.ylabel('female/male [%]',size=15)
plt.savefig('f_m_evolution.png')

In [ ]:
kkk=dfpp[dfpp.year>2000]
kkk[ (kkk.gender == 'male') | (kkk.gender == 'female')].year.value_counts()

https://www.quora.com/How-do-I-find-the-minimum-number-of-votes-for-a-percentage-to-be-true-e-g-63-in-a-poll
Margin of error = 1.96 * sqrt((p * (1-p)) / n)

Where:

1.96 is the critical value for a 95% confidence interval

p is the proportion of upvotes in the sample

(1-p) is the proportion of downvotes in the sample

n is the sample size

For the first post with 40 votes and 28/12 upvotes/downvotes:

p = 28/40 = 0.7

(1-p) = 0.3

n = 40

Plugging these values into the formula, we get:

Margin of error = 1.96 * sqrt((0.7 * 0.3) / 40) = 0.196

So the margin of error for the first post is 19.6%.

In [ ]:
1.96 * sqrt((0.7 * 0.3) / 400) 

In [ ]:
dfpp.shape

### Instution number evolution → Colombia

In [ ]:
kk = dfpp[dfpp['country'] == 'Colombia'].reset_index(drop=True)

In [ ]:
kk.shape

In [ ]:
tppy = kk.year.value_counts().to_dict()

In [ ]:
kk['Total_CO_publications_per_year'] = kk['year'].apply(lambda y: tppy[y])

In [ ]:
ins =pd.DataFrame()
for y in kk.year.sort_values().unique():
    ins = pd.concat( (ins,kk[kk['year'] == y].drop_duplicates('institution_id')) )

In [ ]:
insts = []
dinsts = {}
for y in ins.year.sort_values().unique():
    insts= list(set(insts).union(ins[ins['year']==y].institution.to_list()))
    dinsts[y] = insts

In [ ]:
ins['institutions']= ins.year.apply(lambda y: dinsts[y])

In [ ]:
kkk=ins.drop_duplicates('year').reset_index(drop=True)

In [ ]:
kkk["institutions_number"]=kkk.institutions.apply(len)

In [ ]:
plt.plot(kkk.year,kkk.institutions_number,'ko')
plt.plot(kkk.year,kkk.institutions_number)

### Author count evolution

In [ ]:
kk.shape

In [ ]:
auths = []
dauths = {}
for y in ins.year.sort_values().unique():
    auths= list(set(auths).union(kk[kk['year']==y].author_id.to_list()))
    dinsts[y] = auths

In [ ]:
kk['authors']= kk.year.apply(lambda y: dinsts[y])

In [ ]:
kkkk = kk.drop_duplicates('year').reset_index(drop=True)
kkkk = kkkk.sort_values('year')

In [ ]:
kkkk["authors_number"]=kkkk.authors.apply(len)

In [ ]:
plt.plot(kkkk.year,kkkk.authors_number,'ko')
plt.plot(kkkk.year,kkkk.authors_number)

In [ ]:
kkk[[     'year','institutions','institutions_number']].merge(
    kkkk[['year','authors',      'authors_number']],on='year',how='left'
     ).to_excel('hepcolombia.xlsx',index='False')

## Country by country analysis

In [ ]:
dfpp=dfpp.drop_duplicates(subset=[
     'recid','year','citation_count','journal_id','author_id','institution_id','rank','start_date','end_date',
    'institution','country','current'
    ]).reset_index(drop=True)

In [ ]:
def get_region(c,country='Colombia',LA_countries=['Brazil', 'Mexico', 'Argentina','Chile', 'Colombia','Bolivia','Cuba', #5
              'Costa Rica', 'Ecuador', 'El Salvador', 'Guatemala', 'Honduras', #10
               'Nicaragua', 'Panama', 'Paraguay', 'Peru', #15
              'Dominican Republic','Uruguay','Venezuela']):
    if c in LA_countries:
        region='LA'
    else:
        region='WW'
    if c==country:
        region=pycc.country_name_to_country_alpha2(country)
    return region

In [ ]:
country='Colombia'
dfc=dfp[dfp['country']==country]
au=dfc['author_id'].unique()
dfppc=dfpp[dfpp['author_id'].apply(lambda a: a in au)].drop_duplicates(subset=['institution_id','rank','start_date','end_date']).reset_index(drop=True)

dfppc['region']=dfppc['country'].apply(lambda c:get_region(c,country=country ))

In [ ]:
dfppc

In [ ]:
dfppc['region'].value_counts()

Explanation of next result for PHD: 
* 8 authors who have some rank from Colombia, published as PHD students from Colombia, 
* 21 authors who have some rank from Colombia, published as PHD students from the rest of the Latin America,
* 9 authors who have some rank from Colombia, published as PHD students from the rest of the world wide.

In [ ]:
%pylab inline

In [ ]:
def get_bottom(L):
    xtot=0
    bottom=[]
    for x in L:
        bottom.append(xtot)
        xtot=xtot+x
    return bottom

def get_colors(indexes):
    return [colors.get(x) for x in indexes ]

In [ ]:
CO=pycc.country_name_to_country_alpha2(country)
colors = {f'{CO}':'b','LA':'yellow','WW':'r'}

ranks=['UNDERGRADUATE','PHD','POSTDOC','JUNIOR','SENIOR']
x=0
rank=ranks[x]

vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc),label=vc.keys())

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

plt.xticks(range(5),ranks)
plt.ylabel( f'authors with any {country} rank and specific $x$-axis rank' )

plt.legend()

In [ ]:
dfppc[dfppc['rank']=='UNDERGRADUATE']['region'].value_counts().sort_index()

The 8 authors who have some rank from Colombia, have published as PHD students from Colombia:

In [ ]:
dfppc[ (dfppc['rank']=='PHD') & (dfppc['region']=='CO') ]

In [ ]:
country='Brazil'
dfc=dfp[dfp['country']==country]
au=dfc['author_id'].unique()
dfppc=dfpp[dfpp['author_id'].apply(lambda a: a in au)].drop_duplicates(subset=['institution_id','rank','start_date','end_date']).reset_index(drop=True)

dfppc['region']=dfppc['country'].apply(lambda c:get_region(c,country=country ))

In [ ]:
CO=pycc.country_name_to_country_alpha2(country)
colors[CO]='b'

ranks=['UNDERGRADUATE','PHD','POSTDOC','JUNIOR','SENIOR']
x=0
rank=ranks[x]

vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc),label=vc.keys())

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

plt.xticks(range(5),ranks)
plt.ylabel( f'$x$-axis rank of the author with any {country} rank' )

plt.legend()

In [ ]:
country='Chile'
dfc=dfp[dfp['country']==country]
au=dfc['author_id'].unique()
dfppc=dfpp[dfpp['author_id'].apply(lambda a: a in au)].drop_duplicates(subset=['institution_id','rank','start_date','end_date']).reset_index(drop=True)

dfppc['region']=dfppc['country'].apply(lambda c:get_region(c,country=country ))

In [ ]:
CO=pycc.country_name_to_country_alpha2(country)
colors[CO]='b'

ranks=['UNDERGRADUATE','PHD','POSTDOC','JUNIOR','SENIOR']
x=0
rank=ranks[x]

vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc),label=vc.keys())

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

plt.xticks(range(5),ranks)
plt.ylabel( f'$x$-axis rank of the author with any {country} rank' )

plt.legend()

### Any Colombian paper

In [ ]:
dfp=pd.DataFrame(df)

In [ ]:
country='Colombia'

papers_ranks = [d.get('papers') for d in df if d.get('papers') and 
                        any([True for dd in d.get('papers') if isinstance(dd.get('rank'),str)])]
papers_ranks = flatten_list(papers_ranks)
papers_ranks = [p for p in papers_ranks if p.get('rank')]

dfpp=pd.DataFrame(papers_ranks)
dfpp=dfpp.drop_duplicates(subset=[
     'recid','year','citation_count','journal_id','author_id','institution_id','rank','start_date','end_date','institution','country','current'
    ]).reset_index(drop=True)

dfc=dfp[dfp['country']==country]
au=dfc['author_id'].unique()
dfppc=dfpp[dfpp['author_id'].apply(lambda a: a in au)].drop_duplicates(subset=['institution_id','rank','start_date','end_date']).reset_index(drop=True)

dfppc['region']=dfppc['country'].apply(lambda c:get_region(c,country=country ))

CO=pycc.country_name_to_country_alpha2(country)
colors[CO]='b'

ranks=['UNDERGRADUATE','PHD','POSTDOC','JUNIOR','SENIOR']
x=0
rank=ranks[x]

vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc),label=vc.keys())

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

plt.xticks(range(5),ranks)
plt.ylabel( f'$x$-axis rank of the author with {country} paper' )

plt.legend()

We can identify visitors as JUNIOR/SENIOR in an institution (country) in papers with affiliations to other institutions (countries)

In [ ]:
country='Brazil'

papers_ranks = [d.get('papers') for d in df if d.get('papers') and 
                        any([True for dd in d.get('papers') if isinstance(dd.get('rank'),str)])]
papers_ranks = flatten_list(papers_ranks)
papers_ranks = [p for p in papers_ranks if p.get('rank')]

dfpp=pd.DataFrame(papers_ranks)
dfpp=dfpp.drop_duplicates(subset=[
     'recid','year','citation_count','journal_id','author_id','institution_id','rank','start_date','end_date','institution','country','current'
    ]).reset_index(drop=True)

dfc=dfp[dfp['country']==country]
au=dfc['author_id'].unique()
dfppc=dfpp[dfpp['author_id'].apply(lambda a: a in au)].drop_duplicates(subset=['institution_id','rank','start_date','end_date']).reset_index(drop=True)

dfppc['region']=dfppc['country'].apply(lambda c:get_region(c,country=country ))

CO=pycc.country_name_to_country_alpha2(country)
colors[CO]='b'

ranks=['UNDERGRADUATE','PHD','POSTDOC','JUNIOR','SENIOR']
x=0
rank=ranks[x]

vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc),label=vc.keys())

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

plt.xticks(range(5),ranks)
plt.ylabel( f'$x$-axis rank of the author with {country} paper' )

plt.legend()

In [ ]:
country='Chile'

papers_ranks = [d.get('papers') for d in df if d.get('papers') and 
                        any([True for dd in d.get('papers') if isinstance(dd.get('rank'),str)])]
papers_ranks = flatten_list(papers_ranks)
papers_ranks = [p for p in papers_ranks if p.get('rank')]

dfpp=pd.DataFrame(papers_ranks)
dfpp=dfpp.drop_duplicates(subset=[
     'recid','year','citation_count','journal_id','author_id','institution_id','rank','start_date','end_date','institution','country','current'
    ]).reset_index(drop=True)

dfc=dfp[dfp['country']==country]
au=dfc['author_id'].unique()
dfppc=dfpp[dfpp['author_id'].apply(lambda a: a in au)].drop_duplicates(subset=['institution_id','rank','start_date','end_date']).reset_index(drop=True)

dfppc['region']=dfppc['country'].apply(lambda c:get_region(c,country=country ))

CO=pycc.country_name_to_country_alpha2(country)
colors[CO]='b'

ranks=['UNDERGRADUATE','PHD','POSTDOC','JUNIOR','SENIOR']
x=0
rank=ranks[x]

vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc),label=vc.keys())

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

plt.xticks(range(5),ranks)
plt.ylabel( f'$x$-axis rank of the author with {country} paper' )

plt.legend()

In [ ]:
country='Guatemala'

papers_ranks = [d.get('papers') for d in df if d.get('papers') and 
                        any([True for dd in d.get('papers') if isinstance(dd.get('rank'),str)])]
papers_ranks = flatten_list(papers_ranks)
papers_ranks = [p for p in papers_ranks if p.get('rank')]

dfpp=pd.DataFrame(papers_ranks)
dfpp=dfpp.drop_duplicates(subset=[
     'recid','year','citation_count','journal_id','author_id','institution_id','rank','start_date','end_date','institution','country','current'
    ]).reset_index(drop=True)


au=dfp['author_id'].unique()
dfppc=dfpp[dfpp['author_id'].apply(lambda a: a in au)].drop_duplicates(subset=['institution_id','rank','start_date','end_date']).reset_index(drop=True)

dfppc['region']=dfppc['country'].apply(lambda c:get_region(c,country=country ))

CO=pycc.country_name_to_country_alpha2(country)
colors[CO]='b'

ranks=['UNDERGRADUATE','PHD','POSTDOC','JUNIOR','SENIOR']
x=0
rank=ranks[x]

vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc),label=vc.keys())

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

x += 1
rank=ranks[x]
vc=dfppc[dfppc['rank']==rank]['region'].value_counts().sort_index()
print(vc)
plt.bar(x,vc,color=get_colors(vc.index),bottom=get_bottom(vc))

plt.xticks(range(5),ranks)
plt.ylabel( f'$x$-axis rank of the author' )

plt.legend()

### How many students turn-out in junior-senior

In [ ]:
DF=pd.DataFrame(df)
DF.drop_duplicates(subset=['author_id']).shape

In [ ]:
DF=DF[DF.positions.apply(lambda L: [d.get('rank') for d in L if d.get('rank')] if isinstance(L,list) else []
                         ).apply(len)>0]
DF.shape

Unique authors with declarated profile

In [ ]:
DF=DF.drop_duplicates(subset=['author_id'])
DF.shape

Never JUNIOR or SENIOR

In [ ]:
tmp=DF[DF.positions.apply(lambda L: [d.get('rank') for d in L if d.get('rank')]  ).apply(lambda s: {'JUNIOR', 'SENIOR'}.difference(s)== {'JUNIOR', 'SENIOR'})]
tmp.shape

JUNIOR OR SENIOR

In [ ]:
tmp=DF[DF.positions.apply(lambda L: [d.get('rank') for d in L if d.get('rank')]  ).apply(lambda s: {'JUNIOR', 'SENIOR'}.difference(s)!= {'JUNIOR', 'SENIOR'})]
tmp.shape

JUNIOR OR SENIOR after declarated degree

In [ ]:
tmp[tmp.positions.apply(lambda L: [d.get('rank') for d in L if d.get('rank')]  
                   ).apply(lambda s: {'PHD', 'MASTER','UNDERGRADUATE'}.intersection(s) ).apply(len)>0].shape

Per country

In [ ]:
c='Colombia'
DF=DF[DF.country==c].drop_duplicates(subset=['author_id'])
DF['ranks']=DF.positions.apply(lambda L: [d.get('rank') for d in L if d.get('rank')])
print('TOTAl',DF.shape)
tmp=DF[DF.positions.apply(lambda L: [d.get('rank') for d in L if d.get('rank')]  ).apply(lambda s: {'JUNIOR', 'SENIOR'}.difference(s)== {'JUNIOR', 'SENIOR'})]
print('Never JUNIOR or SENIOR',tmp.shape)

tmp=DF[DF.positions.apply(lambda L: [d.get('rank') for d in L if d.get('rank')]  ).apply(lambda s: {'JUNIOR', 'SENIOR'}.difference(s)!= {'JUNIOR', 'SENIOR'})]
tmp[tmp.positions.apply(lambda L: [d.get('rank') for d in L if d.get('rank')]  
                   ).apply(lambda s: {'PHD', 'MASTER','UNDERGRADUATE'}.intersection(s) ).apply(len)>0].shape
print('JUNIOR OR SENIOR after declarated degree',tmp.shape)

In [ ]:
DF[['full_name','ranks']][50:100]

## Fix bad `profile_id` for `LOCAL → True`

In [ ]:
#author_id='J.S.Useche.1'
def get_profile_id(author_id,sleep=0.4):
    #if True:
    status=False
    l=[]
    pi=None
    #try:
    #if True:
    try:
        #https://inspirehep.net/literature?q=ea+J.D.Ruiz.Alvarez.2+and+ac+1->10
        url=f'https://inspirehep.net/api/literature?q=ea+{author_id}+and+ac+1->10'
        r=requests.get(url)
        time.sleep(sleep)
        if r.status_code==200:
            data=r.json()
            status=True
            #if not `try` fails and `pass``.
        for d in data.get('hits').get('hits'):
            #l=[a.get('record') for a in d.get('metadata').get('authors') if a.get('bai')==author_id]
            try: #failsafe
                l=[a for a in data.get('hits').get('hits')[0].get('metadata').get('authors') if a.get('bai')==author_id]
            except:
                l=[]
        if len(l)==1 and l[0].get('record'):
            pi=l[0].get('record').get('$ref').split('/')[-1]
    except:
        pass
    return pi,status

TEST=True
if TEST:
    assert get_profile_id('J.S.Useche.1')==('2572877',True)
    assert get_profile_id('J.D.Ruiz.Alvarez.2')==('1184534',True)
    assert get_profile_id('S.Das.8')==(None,True)
    assert get_profile_id('M.Sanchez.39')==(None, True)

In [ ]:
get_profile_id('J.S.Useche.1')

In [ ]:
def get_advisor(i,sleep=0.4):
    adv=[]
    try:
        r=requests.get('https://inspirehep.net/api/authors/{}'.format(i))
        time.sleep(sleep)
        adv=r.json().get('metadata').get('advisors')
        if adv is None:
            adv=[]
    except:
        pass
    return adv

In [ ]:
#a=df[0]
#a.get('author_id')
GET_ADVISORS=False
if GET_ADVISORS:
    kk=[a.update({'advisors':get_advisor(a.get('profile_id'))}) for a in df if not a.get('advisors')]

In [ ]:
SAVE=False
status=0
step=1000
i=0
for start in range(0,len(df),step):
    for d in df[start:start+step]:
        i+=1
        print(str(i).zfill(5),status,end='\r')

        if len(set([dd.get('author_id') for dd in df  if dd.get('profile_id')==d.get('profile_id')]))==1:
            continue
        author_id=d.get('author_id')
        check_profile_id=get_profile_id(author_id)
        if check_profile_id[1] and check_profile_id[0]!=d.get('profile_id'):
            d['profile_id']=check_profile_id[0]
            d['advisors']=get_advisor(d.get('profile_id'))
            status=1
        else:
            status=0
    if SAVE:
        f=open('data/inspire_LA.json','w')
        json.dump(df,f)
        f.close()

## Students object

In [ ]:
[dd for dd in df if dd.get('author_id')=='M.Sanchez.39']

In [ ]:
df=pd.DataFrame(df)

In [ ]:
df[df.advisors.apply(len)>0].shape

In [ ]:
df[ (df['positions'].str[0].str['rank'].apply(lambda r: r is not None)) & 
   (df['positions'].str[0].str['rank'].apply(lambda r: r is not None)) ].shape

In [ ]:
df[df['positions'].str[0].apply(lambda r:isinstance(r,dict))].shape

People which have edited their profile in INSPIREHEP

In [ ]:
df[ (df['positions'].str[0].str['rank'].apply(lambda r:isinstance(r,str))) 
    | (df.advisors.apply(len)>0 ) ].shape

In [ ]:
df.shape

In [ ]:
#df['profile_id']=df['profile_id'].fillna(0).astype(int).astype(str).apply(lambda p: None if p=='0' else p)

In [ ]:
df[df['profile_id']=='991924']

In [ ]:
L=[1,2,3]
L.remove(1)
L

In [ ]:
profile_id = '991924'
a = {'ids': [{'value': 'INSPIRE-00133260', 'schema': 'INSPIRE ID'}],
 'name': 'Valle, Jose W.F.',
 'hidden': False,
 'record': {'$ref': 'https://inspirehep.net/api/authors/985058'},
 'degree_type': 'phd',
 'curated_relation': False}
r = {'author_id':'D.Restrepo.1',
   'name':{'value': 'Restrepo Quintero, Diego Alejandro',
   'preferred_name': 'Diego Restrepo'},
   'positions': [{'rank': 'PHD',
                  'hidden': False,
                  'record': {'$ref': 'https://inspirehep.net/api/institutions/907907'},
                  'current': False,
                  'end_date': '2001',
                  'start_date': '1997',
                  'institution': 'Valencia U., IFIC',
                  'curated_relation': True}
                ]
   }
advisor_profile_id = '985058'
d = {}

def get_position(ds, p):
    for k in p.keys():
        if k == 'record':
            try:
                ds['institution_id'] = p[k].get('$ref').split('/')[-1]
            except:
                ds['institution_id'] = None
        else:
            ds[k] = p[k]
    return ds


def get_student(rr, a, profile_id,ranks=['UNDERGRADUATE','MASTER','PHD']):
    "Trigger if advisor exits within `a`"
    r=rr.copy()
    old_ranks=ranks.copy()
    students=[]
    degree_type = a.get('degree_type')
    if degree_type=='bachelor':
        degree_type='undergraduate'
    if not r.get('positions'):
        r['positions']=[]
    #    return students
    for p in r.get('positions'):
        ds = {}
        if p.get('rank') and p.get('rank').lower() == degree_type:
            ds = get_position(ds,p)
            if old_ranks and p.get('rank') in old_ranks:
                old_ranks=old_ranks.remove(p.get('rank'))
        elif (isinstance(r.get('advisors'),list) and len(r.get('advisors'))==1 and p.get('rank') and 
              old_ranks and p.get('rank') in old_ranks and (p.get('rank')=='PHD' or p.get('rank')=='MASTER')):
            ds = get_position(ds,p) 
        else:
            continue
        if ds:
            if ds.get('rank') not in ranks and degree_type:
                ds['rank'] = degree_type.upper()
            ds['student_id'] = r.get('author_id')
            ds['student_name'] = r.get('name').get('value')
            ds['student_profile_id'] = profile_id
            students.append(ds)
    # if not positons in ranks
    if not students:
        #Advisor declared but not in positions
        ds = {}
        ds['rank'] = "PHD"
        if r.get('author_id'):
            ds['student_id'] = r.get('author_id')
            if r.get('name'):
                ds['student_name'] = r.get('name').get('value')
            ds['student_profile_id'] = profile_id
        students.append(ds)


    return students
#assert get_student(r, a, profile_id).get('institution_id') == '907907'

def get_student_without_advisor(p, r, profile_id, ranks=['UNDERGRADUATE','MASTER','PHD']):
    ds = {}
    ds = get_position(ds, p)
    if r.get('author_id'):
        ds['student_id'] = r.get('author_id')
        if r.get('name'):
            ds['student_name'] = r.get('name').get('value')
        ds['student_profile_id'] = profile_id
    return ds


def get_id(d, df, id_type='profile_id'):
    new_id = d.get('profile_id')
    try:
        lpid = list(set(df[ (df['name'].str['value']==d.get('name')) ][id_type].to_list()))
    except:
        lpid=[]
    if len(lpid) == 1:
        if lpid[0]:
            new_id = lpid[0]
    return new_id


def get_advisor(d, advisor_profile_id, a, df):
    d['profile_id'] = advisor_profile_id

    d['name'] = a.get('name')

    if not d.get('profile_id'):
        d['profile_id'] = get_id(d, df, id_type='profile_id')
    if not d.get('profile_id'):
        d['author_id'] = get_id(d, df, id_type='author_id')

    return d
assert get_advisor(d,advisor_profile_id, a, df).get('profile_id') == '985058'

In [ ]:
# 991924  # J Valle advisor
# 1854904 # not advisor → Unknown
# 1045947 # not position
# 1014628 # J Valle advisor
# 1028906 # not advisor → Unknown
profile_id=['991924','1014628','1854904','1045947','1021727','1073921','1066143','989075','983327','1009735','1402655','1078540'][-1]
r=df[df['profile_id']==profile_id].iloc[0].to_dict()

In [ ]:
f=open(f'tests/data/{profile_id}.json','w')
json.dump(r,f)
f.close()

In [ ]:
r.get('advisors')[0]

In [ ]:
r.get('positions')

In [ ]:
def get_students(r, profile_id, pre_students=[]):
    ranks = ['UNDERGRADUATE', 'MASTER', 'PHD']

    if any([ any([ss.get('student_profile_id') == profile_id for ss in s.get('students')])
       for s in pre_students]):
        print('Already in students')
        #raise Exception('Already in students')
        return pre_students

    for a in r.get('advisors'):
        # Get profile_id or author_id or name of advisor
        try:
            advisor_profile_id = a.get('record').get('$ref').split('/')[-1]
        except:
            advisor_profile_id = ''

        OLD_ADVISOR = False
        for old_a in pre_students:
            # TODO: check for other indetifiers
            if advisor_profile_id and old_a.get('profile_id') == advisor_profile_id:
                old_a['students'].extend(get_student(r, a, profile_id,ranks))
                OLD_ADVISOR = True
                break
        if OLD_ADVISOR:
            continue

        d = {}
        d = get_advisor(d, advisor_profile_id, a, df)

        d['students'] = get_student(r, a, profile_id,ranks)

        pre_students.append(d)



    # Update ranks
    if r.get('advisors'):
        L = [[s.get('rank') for s in a.get('students') if s.get('rank')] for a in pre_students]
        ranks = set(ranks).difference([item for sublist in L for item in sublist])

    if not r.get('positions'):
        #raise Exception('Already in students')
        return pre_students


    for p in r.get('positions'):
        if p.get('rank') in ranks:
            d = {}
            d['students'] = []
            ds = get_student_without_advisor(p, r, profile_id, ranks=ranks)
            d['students'].append(ds)
            d['profile_id'] = None
            d['name'] = 'Unkown'
            pre_students.append(d)

    return pre_students


In [ ]:
profile_id='991924'
f=open(f'tests/data/{profile_id}.json','r')
r=json.load(f)
f.close()
pre_students = get_students(r, profile_id, [])
assert len(pre_students)==3
profile_id='1014628'
f=open(f'tests/data/{profile_id}.json','r')
r=json.load(f)
f.close()
pre_students = get_students(r, profile_id, [])
assert pre_students[0].get('students')[0].get('rank')=='PHD'
profile_id='1854904'
f=open(f'tests/data/{profile_id}.json','r')
r=json.load(f)
f.close()
pre_students = get_students(r, profile_id, [])
assert len(pre_students)==3
profile_id='1045947'
f=open(f'tests/data/{profile_id}.json','r')
r=json.load(f)
f.close()
pre_students = get_students(r, profile_id, [])
assert pre_students[0].get('profile_id')=='991924'
profile_id='1073921'
f=open(f'tests/data/{profile_id}.json','r')
r=json.load(f)
f.close()
pre_students = get_students(r, profile_id, [])
assert len(set(['MASTER','PHD']).difference(
    [[dd.get('rank') for dd in d.get('students')] for d in pre_students][0]
 ))==0
profile_id='1066143'
f=open(f'tests/data/{profile_id}.json','r')
r=json.load(f)
f.close()
LIST=[[dd.get('rank') for dd in d.get('students')] for d in get_students(r,profile_id, [])]
assert [item for sublist in LIST for item in sublist] == ['PHD', 'PHD']

In [ ]:
dfa=df.drop_duplicates(subset=['profile_id']).reset_index(drop=True)

In [ ]:
pre_students = []
for profile_id in dfa['profile_id']:
    r = df[df['profile_id']==profile_id]
    if not r.empty:
        r = r.iloc[0].to_dict()
    else:
        continue
    if profile_id and r:
        pre_students = get_students(r, profile_id,pre_students)

Get Advisor level

In [6]:
def add_level(pre_students,LL,level=0):
    for parent_id in LL:
        #print(parent_id,end='\r')
        try:
            d=[d for d in pre_students if d.get('profile_id')==parent_id][0]
            d['level']=level
        except:
            continue

In [7]:
ps=pd.DataFrame(pre_students)
ps=ps.dropna(subset=['profile_id']).reset_index(drop=True)
# students
L=ps.students.apply(lambda L: [d.get('student_profile_id') for d in L]).to_list()
# Level 0 advisors
LL=list(  set(ps.profile_id.to_list()).difference( set([item for subitem in L for item in subitem]) ) ) 
LL=[x for x in LL if x]
print(len(LL))

level=0
while LL:
    add_level(pre_students,LL,level=level)

    LL=[ [dd.get('student_profile_id') for dd in  d.get('students') if dd.get('rank')=='PHD'] for d in pre_students if d.get('level')==level]
    LL=set([item for sublist in LL for item in sublist])
    LL=[x for x in LL if x]
    print(level,len(LL))
    level+=1
    if level>10:
        break

NameError: name 'pre_students' is not defined

In [ ]:
chk_levels=['1020782','1013015','989075','996018','1026011','1066143']
LL=[[d.get('level') for d in pre_students if d.get('profile_id')==chk_levels[i]] for i in range(len(chk_levels))]
assert [item for sublist in LL for item in sublist]==list(range(6))

In [ ]:
[d for d in pre_students if d.get('level')==4][0]

In [ ]:
f=open('data/students.json','w')
json.dump(pre_students,f)
f.close()

In [ ]:
student_profile_id='1045947'
[d for d in pre_students if any([dd.get('student_profile_id')==student_profile_id for dd in d.get('students')])]

Example: CINVESTAV-IPN

In [ ]:
institution_id='903002' #'907907' # '903002' # '903906'
ins_studs=[d for d in pre_students if any([dd.get('institution_id')==institution_id for dd in d.get('students')])]

In [ ]:
cinvestav=[]
for d in ins_studs:
    dd=d.copy()
    students=[]
    for s in dd.get('students'):
        if s.get('institution_id')==institution_id:
            students.append(s)
    dd['students']=students
    cinvestav.append(dd)


In [ ]:
LL=[d.get('students') for d in cinvestav]
len([item for sublist in LL for item in sublist])

In [ ]:
cv=pd.DataFrame(cinvestav)
LL=cv.apply(lambda d: [{'student_name':dd.get('student_name'),
                     'profile_id':dd.get('student_profile_id'),
                     'degree':dd.get('rank'),
                     'start':dd.get('start_date'),
                     'end':dd.get('end_date'),#'student_position':dd.get('position'),
                     'advisor_name':d.get('name'),
                     'advisor_id':d.get('profile_id')} for dd in d.get('students')],axis=1)
LL=[item for sublist in LL for item in sublist]
cv_df=pd.DataFrame(LL)
cv_df=cv_df.sort_values(by=['advisor_name','end']).reset_index(drop=True)
cv_df[:1]

In [ ]:
cv_df=cv_df.merge(df[['institution','profile_id']],on='profile_id',how='left').drop_duplicates(
    ['student_name','degree','advisor_name']).rename(columns={'institution':'position'}).reset_index(drop=True)
cv_df    

In [ ]:
cv_df[cv_df['advisor_name'].str.contains('Roig')]

In [ ]:
cv_df.to_csv('data/cinvestav.csv',index=False)

In [ ]:
cv_df[cv_df['advisor_name'].str.contains('Zepeda')]

## Get all the institutions from INSPIREHEP

In [ ]:
# united states, england, germany, france, italy, china, spain, japan, korea
# Missing: Caltech; Princeton U.; Valencia U., IFIC; Oxford U.; Cambridge U., DAMTP; Aachen, Tech. Hochsch.; Humboldt U., Berlin; DESY; CERN; Fermilab; SLAC; Heidelberg U.
# Munich, Max Planck Inst.; Hamburg U.; Mainz U., Inst. Phys.; Frankfurt U.; Munich, Tech. U.; Imperial Coll., London; INFN, Turin; INFN, Rome; SISSA, Trieste; Rome U.; 
# Frascati; ICTP, Trieste; Turin U.
def get_institutions(q):
    page=1
    if q:
        url=f'https://inspirehep.net/api/institutions?q={q}&size=250&page={page}'
    else:
        url=f'https://inspirehep.net/api/institutions?size=250&page={page}' 
    r=requests.get(url)
    time.sleep(sleep)
    i=r.json()['hits']['hits']
    total=r.json().get('hits').get('total')
    if not page or page==1:
        ins=[d.get('metadata') for d in i]
    j=page
    next=r.json().get('links').get('next')
    while next:
        print(j)
        r=requests.get(next)
        time.sleep(sleep)
        try:
            i=r.json()['hits']['hits']
        except:
            i=[]
        ins.extend([d.get('metadata') for d in i])
        try:
            next=r.json().get('links').get('next')
        except:
            next=None
        j+=1
        if j > total//250:
            break
    return ins



In [ ]:
kk=[]
for q in ['','u','england','germany','france','italy','spain','korea','china','japan','united states',
    'russia','brazil']:
    print(q)
    ins=get_institutions(q)
    for d in ins:
        if d not in kk:
            kk.append(d)
    print(len(kk))
    if len(kk)>11835:
        break            


In [ ]:
ins=kk.copy()

In [ ]:
f=open('data/inspire_institutions.json','r')
old=json.load(f)
f.close()

In [ ]:
for d in old:
    if d not in kk:
        kk.append(d)

In [ ]:
cn=[d.get('control_number') for d in kk]

In [ ]:
duplicates=list(set([x for x in cn if cn.count(x)>1]))
len(duplicates)


In [ ]:
len(set([d.get('control_number') for d in kk]))

In [ ]:
ins=[]
for d in kk:
    if d.get('control_number') not in duplicates:
        ins.append(d)

In [ ]:
for cn in duplicates:
    ins.append( [d for d in  kk if d.get('control_number')==cn][-1] )

In [ ]:
len(ins)

In [ ]:
kk=ins.copy()

In [ ]:
f=open('data/inspire_institutions.json','w')
json.dump(kk,f)
f.close()

In [ ]:
dfi=pd.DataFrame(kk)
dfi=dfi.sort_values('number_of_papers', ascending=False).reset_index(drop=True)

In [ ]:
def not_cn(name,sleep=0.4):
    url=f'https://inspirehep.net/api/literature?q=aff+{name}+and+ac+1->10&size=1'
    r=requests.get(url)
    time.sleep(sleep)
    try:
        total=r.json().get('hits').get('total')
    except:
        total=None
    return total

In [ ]:
dfi['number_of_not_cn_papers']=dfi.legacy_ICN.apply(not_cn)

In [ ]:
dfi['number_of_not_cn_papers']=dfi.number_of_not_cn_papers.fillna(-1).astype(int)

In [ ]:
dfi.sort_values(['number_of_not_cn_papers','number_of_papers'], ascending=False)[['legacy_ICN','number_of_not_cn_papers','number_of_papers']][0:50].reset_index(drop=True)#.to_csv('kk.csv',index=False)

In [ ]:
control_number=917747
dfi[dfi['control_number']==control_number].iloc[0].number_of_not_cn_papers

In [ ]:
for d in ins:
    d['number_of_not_cn_papers']=int(dfi[dfi['control_number']==d.get('control_number')].iloc[0].number_of_not_cn_papers)

In [ ]:
f=open('data/inspire_institutions.json','w')
json.dump(ins,f)
f.close()

In [ ]:
kkk=pd.DataFrame(kk)
kkk[kkk.legacy_ICN.str.lower().str.contains('valencia')]

In [ ]:
DF = pd.DataFrame(df)
DF[DF['advisors'].apply(len)>0].shape